In [1]:
pip install minisom

In [2]:
pip install chardet

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tslearn scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
import json
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
import ast
from sklearn.decomposition import PCA
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.clustering import TimeSeriesKMeans
from minisom import MiniSom
from sklearn.cluster import KMeans

In [5]:
json_file = "C:\\Users\\tigra\\OneDrive\\Documents\\cleaned_df.jsonl"

In [6]:
timestamps = []
channel_x = []
channel_y = []

with open(json_file, 'r') as file:
    for line in file:
        try:
            data = json.loads(line)
            timestamps.append(data['timestamp'])
            channel_x.append(data['channel_x'])
            channel_y.append(data['channel_y'])
            #print(data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")


time_series_data = pd.DataFrame({
    'timestamp': timestamps,
    'channel_x': channel_x,
    'channel_y': channel_y
})

In [7]:
time_series_data

,timestamp,channel_x,channel_y
0,9:39:39,"[0.8860844827, 0.2398780831, -0.7588045344, -1...","[-1.0970508836, 1.0021835283, 0.5548057028, 0...."
1,9:39:49,"[-0.3132895666, -0.0143246711, 0.0585480222, -...","[0.7455526429, -0.9193375962, 0.9783047733, -0..."
2,9:39:59,"[-0.1941087483, -0.2919947281, -0.6816562247, ...","[0.3501117451, -0.4537091558, -1.3679966717, -..."
3,9:40:9,"[0.0933388765, -0.0111775116, 0.0284666356, -0...","[-2.1192795178, 0.4437364412, -0.4180832858, 0..."
4,9:40:19,"[-0.3346322388, 0.2124313304, 0.568905011, 0.8...","[2.0316917182, 0.3680117201, -0.7623467467, -0..."
...,...,...,...
2798,17:25:59,"[0.0571303255, 0.0167320325, -0.1312956609, -0...","[-0.2585808797, 0.4511690181, 1.3718370909, -0..."
2799,17:26:9,"[0.7842501127, 0.6485894692, 0.884297264, 0.24...","[-0.8001867618, -0.6901360987, 0.2612386972, 1..."
2800,17:26:19,"[0.542245385, 0.9750843589, 0.6135731943, 0.73...","[0.0708408103, -0.585789414, -0.2912445529, 0...."
2801,17:26:29,"[-0.1005815044, 0.1743217626, 0.6969714747, 0....","[-1.461029906, 0.1401113917, 2.7590324291, 0.3..."


In [8]:
#x_values = np.arange(1, len(time_series_data['channel_x'].iloc[0]) + 1)
#y_values = np.array(time_series_data['channel_x'].iloc[0])

In [9]:
def som_kmeans_clustering(data, som_dim, num_clusters, sigma, num_iteration):

    reshaped_data = np.array([np.array(time_series) for time_series in data])

    # Train SOM
    som = MiniSom(x=som_dim, y=som_dim, input_len=reshaped_data.shape[1], sigma=sigma, learning_rate=0.5)
    som.random_weights_init(reshaped_data)
    som.train_random(reshaped_data, num_iteration=num_iteration)

    weights = som.get_weights()
    flattened_weights = weights.reshape(-1, weights.shape[-1])

    # Perform k-means clustering
    kmeans = KMeans(n_clusters=num_clusters, random_state=0)
    kmeans.fit(flattened_weights)

    # Assign clusters
    clusters = kmeans.predict(reshaped_data)
    
    return clusters

In [10]:
som_dim = 10  # SOM grid size (this can be adjusted)
num_clusters = 4
sigma = 0.5 
num_iteration = 10000

In [11]:
channel_x = time_series_data['channel_x'].values

In [13]:
channel_x_clusters = som_kmeans_clustering(channel_x, som_dim, num_clusters, sigma, num_iteration)

C:\Users\tigra\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [40]:
time_series_data['channel_x_cluster'] = channel_x_clusters
time_series_data

,timestamp,channel_x,channel_y,channel_x_cluster
0,9:39:39,"[0.8860844827, 0.2398780831, -0.7588045344, -1...","[-1.0970508836, 1.0021835283, 0.5548057028, 0....",3
1,9:39:49,"[-0.3132895666, -0.0143246711, 0.0585480222, -...","[0.7455526429, -0.9193375962, 0.9783047733, -0...",3
2,9:39:59,"[-0.1941087483, -0.2919947281, -0.6816562247, ...","[0.3501117451, -0.4537091558, -1.3679966717, -...",3
3,9:40:9,"[0.0933388765, -0.0111775116, 0.0284666356, -0...","[-2.1192795178, 0.4437364412, -0.4180832858, 0...",3
4,9:40:19,"[-0.3346322388, 0.2124313304, 0.568905011, 0.8...","[2.0316917182, 0.3680117201, -0.7623467467, -0...",3
...,...,...,...,...
2798,17:25:59,"[0.0571303255, 0.0167320325, -0.1312956609, -0...","[-0.2585808797, 0.4511690181, 1.3718370909, -0...",3
2799,17:26:9,"[0.7842501127, 0.6485894692, 0.884297264, 0.24...","[-0.8001867618, -0.6901360987, 0.2612386972, 1...",3
2800,17:26:19,"[0.542245385, 0.9750843589, 0.6135731943, 0.73...","[0.0708408103, -0.585789414, -0.2912445529, 0....",3
2801,17:26:29,"[-0.1005815044, 0.1743217626, 0.6969714747, 0....","[-1.461029906, 0.1401113917, 2.7590324291, 0.3...",3
